# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [1]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 0.7903232003508388
 2.2080487943252667
 2.046479072975802

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [2]:
Alu = lu(A)

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
3×3 Matrix{Float64}:
 1.0        0.0       0.0
 0.366062   1.0       0.0
 0.586798  -0.913725  1.0
U factor:
3×3 Matrix{Float64}:
 0.791473   0.999758  0.255248
 0.0       -0.254289  0.295474
 0.0        0.0       1.04481

In [3]:
typeof(Alu)

LU{Float64, Matrix{Float64}, Vector{Int64}}

The different parts of the factorization can be extracted by accessing their special properties

In [4]:
Alu.P

3×3 Matrix{Float64}:
 0.0  0.0  1.0
 1.0  0.0  0.0
 0.0  1.0  0.0

In [5]:
Alu.L

3×3 Matrix{Float64}:
 1.0        0.0       0.0
 0.366062   1.0       0.0
 0.586798  -0.913725  1.0

In [6]:
Alu.U

3×3 Matrix{Float64}:
 0.791473   0.999758  0.255248
 0.0       -0.254289  0.295474
 0.0        0.0       1.04481

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [7]:
A\b

3-element Vector{Float64}:
 0.9999999999999994
 1.0000000000000002
 1.0000000000000002

In [8]:
Alu\b

3-element Vector{Float64}:
 0.9999999999999994
 1.0000000000000002
 1.0000000000000002

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [9]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [10]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.301071   0.773468   -0.557767
 -0.482617  -0.628056   -0.610432
 -0.822458   0.0854046   0.562378
R factor:
3×3 Matrix{Float64}:
 -0.962326  -1.25115   -0.773252
  0.0       -0.342613  -0.258097
  0.0        0.0       -0.637786

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [11]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.301071   0.773468   -0.557767
 -0.482617  -0.628056   -0.610432
 -0.822458   0.0854046   0.562378

In [12]:
Aqr.R

3×3 Matrix{Float64}:
 -0.962326  -1.25115   -0.773252
  0.0       -0.342613  -0.258097
  0.0        0.0       -0.637786

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [13]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -1.18054684830493
  0.3567033738339611
  3.5518077270477697
vectors:
3×3 Matrix{Float64}:
 -0.382278   0.844888   -0.374202
 -0.466467  -0.52602    -0.711134
  0.797666   0.0972982  -0.595199

The values and the vectors can be extracted from the Eigen type by special indexing

In [14]:
AsymEig.values

3-element Vector{Float64}:
 -1.18054684830493
  0.3567033738339611
  3.5518077270477697

In [15]:
AsymEig.vectors

3×3 Matrix{Float64}:
 -0.382278   0.844888   -0.374202
 -0.466467  -0.52602    -0.711134
  0.797666   0.0972982  -0.595199

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [16]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
  1.0           3.55271e-15  -4.27436e-15
  1.55431e-15   1.0          -1.69309e-15
 -2.10942e-15  -3.10862e-15   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [17]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [18]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [19]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

0.31080854640633504

In [20]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [21]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [22]:
@time eigvals(Asym);

  0.260466 seconds (268.81 k allocations: 25.902 MiB, 68.80% compilation time)


In [23]:
@time eigvals(Asym_noisy);

  0.470274 seconds (14 allocations: 7.928 MiB)


In [24]:
@time eigvals(Asym_explicit);

  0.129337 seconds (69.82 k allocations: 12.733 MiB, 3.46% gc time, 27.93% compilation time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [25]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.596842 seconds (491.92 k allocations: 214.376 MiB, 1.72% gc time, 21.44% compilation time)


7.087455018812964

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [26]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [27]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 2//5   1//10  1//5
 3//5   1//1   7//10
 7//10  4//5   1//10

In [28]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
  7//10
 23//10
  8//5

In [29]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1//1
 1//1
 1//1

In [30]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}, Vector{Int64}}
L factor:
3×3 Matrix{Rational{BigInt}}:
 1//1    0//1   0//1
 4//7    1//1   0//1
 6//7  -22//25  1//1
U factor:
3×3 Matrix{Rational{BigInt}}:
 7//10   4//5    1//10
 0//1   -5//14   1//7
 0//1    0//1   37//50

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [53]:
using LinearAlgebra
A = [
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]

5×5 Matrix{Int64}:
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36

In [44]:
A_eigv = eigvals(A)

5-element Vector{Float64}:
 -128.49322764802147
  -55.887784553056875
   42.7521672793189
   87.16111477514521
  542.4677301466143

In [45]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

LoadError: AssertionError: A_eigv == [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [79]:
A_diag = Diagonal(A_eigv)
A_diag[1,1]=round(A_diag[1,1]; digits=3);
A_diag[5,5]=round(A_diag[5,5]; digits=3);
A_diag = round.(A_diag;digits=4)

5×5 Diagonal{Float64, Vector{Float64}}:
 -128.493     ⋅        ⋅        ⋅         ⋅ 
     ⋅     -55.8878    ⋅        ⋅         ⋅ 
     ⋅        ⋅      42.7522    ⋅         ⋅ 
     ⋅        ⋅        ⋅      87.1611     ⋅ 
     ⋅        ⋅        ⋅        ⋅      542.468

In [80]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [81]:
A_lowertri = LowerTriangular(A)

5×5 LowerTriangular{Int64, Matrix{Int64}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

In [82]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]